In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [3]:
from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage

"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [4]:
tool = TavilySearchResults(max_results=2)

In [7]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"]
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [8]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-3.5-turbo")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [9]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in SF?', id='b2a9460e-617f-4eae-b67d-e8e9b977fa95'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_aMgdD7AoorYiGWtKrMhiwnqI', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d07c7d64-311c-4bf6-888c-ba469609b710-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_aMgdD7AoorYiGWtKrMhiwnqI'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_aMgdD7AoorYiGWtKrMhiwnqI', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name

In [11]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', id='b2a9460e-617f-4eae-b67d-e8e9b977fa95'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_aMgdD7AoorYiGWtKrMhiwnqI', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d07c7d64-311c-4bf6-888c-ba469609b710-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_aMgdD7AoorYiGWtKrMhiwnqI'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}, next=('action',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef31516-98c0-694c-8001-a2d1fc387e0a'}}, metadata={'source': 'loop', 'step': 1, 'wri

In [12]:
abot.graph.get_state(thread).next

('action',)

In [13]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_aMgdD7AoorYiGWtKrMhiwnqI'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://world-weather.info/forecast/usa/san_francisco/june-2024/\', \'content\': \'Extended weather forecast in San Francisco. Hourly Week 10 days 14 days 30 days Year. Detailed ⚡ San Francisco Weather Forecast for June 2024 - day/night 🌡️ temperatures, precipitations - World-Weather.info.\'}, {\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1719141102, \'localtime\': \'2024-06-23 4:11\'}, \'current\': {\'last_updated_epoch\': 1719140400, \'last_updated\': \'2024-06-23 04:00\', \'temp_c\': 14.4, \'temp_f\': 57.9, \'is_day\': 0, \'condition\': {\'text\': \'Clear\', \'icon\': \'/

In [14]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', id='b2a9460e-617f-4eae-b67d-e8e9b977fa95'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_aMgdD7AoorYiGWtKrMhiwnqI', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d07c7d64-311c-4bf6-888c-ba469609b710-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_aMgdD7AoorYiGWtKrMhiwnqI'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173}), ToolMessage(content='[{\'url\': \'https://world-weather.info/forecast/usa/san_francisco/june-2024/\', \'content\': \'Extended weather forecast in San Francisco. 

In [15]:
abot.graph.get_state(thread).next

()

In [16]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "3"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)
while abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread),"\n")
    _input = input("proceed?")
    if _input != "y":
        print("aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='94270536-f714-429d-b976-15ba664dbeb3'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Og5snzEMjgyX7C7HVBVB22GV', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 152, 'total_tokens': 174}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2c0d1cd2-753e-45af-a27a-b99c6fcb53d9-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_Og5snzEMjgyX7C7HVBVB22GV'}], usage_metadata={'input_tokens': 152, 'output_tokens': 22, 'total_tokens': 174})]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Og5snzEMjgyX7C7HVBVB22GV', 'function': {'arguments': '{"query":"current weather in Los

In [19]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "11"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='f849bee2-8346-45f8-be50-e776397733ab'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EtyP8RyLpKhfXchP8bGovHMv', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-97ec91ac-4a92-4857-bc55-82831097eb09-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_EtyP8RyLpKhfXchP8bGovHMv'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EtyP8RyLpKhfXchP8bGovHMv', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'ta

In [20]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='f849bee2-8346-45f8-be50-e776397733ab'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EtyP8RyLpKhfXchP8bGovHMv', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-97ec91ac-4a92-4857-bc55-82831097eb09-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_EtyP8RyLpKhfXchP8bGovHMv'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}, next=('action',), config={'configurable': {'thread_id': '11', 'thread_ts': '1ef31527-48b2-6e26-8001-30b8efe6b535'}}, metadata={'source': 'loop', 'step': 1, 'writes

In [21]:
current_values = abot.graph.get_state(thread)

In [22]:
current_values

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='f849bee2-8346-45f8-be50-e776397733ab'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EtyP8RyLpKhfXchP8bGovHMv', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-97ec91ac-4a92-4857-bc55-82831097eb09-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_EtyP8RyLpKhfXchP8bGovHMv'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}, next=('action',), config={'configurable': {'thread_id': '11', 'thread_ts': '1ef31527-48b2-6e26-8001-30b8efe6b535'}}, metadata={'source': 'loop', 'step': 1, 'writes

In [23]:
current_values.values['messages'][-1]

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EtyP8RyLpKhfXchP8bGovHMv', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-97ec91ac-4a92-4857-bc55-82831097eb09-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_EtyP8RyLpKhfXchP8bGovHMv'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})

In [24]:
current_values.values['messages'][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'weather in Los Angeles'},
  'id': 'call_EtyP8RyLpKhfXchP8bGovHMv'}]

In [25]:
_id = current_values.values['messages'][-1].tool_calls[0]['id']
current_values.values['messages'][-1].tool_calls = [
    {'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in Louisiana'},
  'id': _id}
]

In [26]:
current_values.values['messages'][-1]

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EtyP8RyLpKhfXchP8bGovHMv', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-97ec91ac-4a92-4857-bc55-82831097eb09-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_EtyP8RyLpKhfXchP8bGovHMv'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})

In [27]:
abot.graph.update_state(thread, current_values.values)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='f849bee2-8346-45f8-be50-e776397733ab'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EtyP8RyLpKhfXchP8bGovHMv', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-97ec91ac-4a92-4857-bc55-82831097eb09-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_EtyP8RyLpKhfXchP8bGovHMv'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}


{'configurable': {'thread_id': '11',
  'thread_ts': '1ef31536-21bb-66f6-8002-bf56c29ebfda'}}

In [28]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='f849bee2-8346-45f8-be50-e776397733ab'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EtyP8RyLpKhfXchP8bGovHMv', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-97ec91ac-4a92-4857-bc55-82831097eb09-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_EtyP8RyLpKhfXchP8bGovHMv'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}, next=('action',), config={'configurable': {'thread_id': '11', 'thread_ts': '1ef31536-21bb-66f6-8002-bf56c29ebfda'}}, metadata={'source': 'update', 'step': 2,

In [29]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_EtyP8RyLpKhfXchP8bGovHMv'}
Back to the model!
{'messages': [ToolMessage(content="[{'url': 'https://world-weather.info/forecast/usa/louisiana/june-2024/', 'content': 'Detailed ⚡ Louisiana Weather Forecast for June 2024 - day/night 🌡️ temperatures, precipitations - World-Weather.info. Add the current city. Search. Weather; Archive; Widgets °F. World; United States; Missouri; Weather in Louisiana; ... 23 +82° +70° 24 +82° +70° 25 ...'}, {'url': 'https://world-weather.info/forecast/usa/new_orleans/june-2024/', 'content': 'Extended weather forecast in New Orleans. Hourly Week 10 days 14 days 30 days Year. Detailed ⚡ New Orleans Weather Forecast for June 2024 - day/night 🌡️ temperatures, precipitations - World-Weather.info.'}]", name='tavily_search_results_json', tool_call_id='call_EtyP8RyLpKhfXchP8bGovHMv')]}
{'messages': [HumanMessage(content='Whats the weather in LA?', id='f849be

In [30]:
states = []
for state in abot.graph.get_state_history(thread):
    print(state)
    print('--')
    states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='f849bee2-8346-45f8-be50-e776397733ab'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EtyP8RyLpKhfXchP8bGovHMv', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-97ec91ac-4a92-4857-bc55-82831097eb09-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_EtyP8RyLpKhfXchP8bGovHMv'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173}), ToolMessage(content="[{'url': 'https://world-weather.info/forecast/usa/louisiana/june-2024/', 'content': 'Detailed ⚡ Louisiana Weather Forecast for June 2024 -

In [31]:
to_replay = states[-3]

In [32]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='f849bee2-8346-45f8-be50-e776397733ab'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EtyP8RyLpKhfXchP8bGovHMv', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-97ec91ac-4a92-4857-bc55-82831097eb09-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_EtyP8RyLpKhfXchP8bGovHMv'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}, next=('action',), config={'configurable': {'thread_id': '11', 'thread_ts': '1ef31527-48b2-6e26-8001-30b8efe6b535'}}, metadata={'source': 'loop', 'step': 1, 'writes

In [33]:
for event in abot.graph.stream(None, to_replay.config):
    for k, v in event.items():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_EtyP8RyLpKhfXchP8bGovHMv'}
Back to the model!
{'messages': [ToolMessage(content="[{'url': 'https://www.wunderground.com/hourly/us/ca/los-angeles/90024/date/2024-6-30', 'content': 'Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the Los Angeles area. ... 34.06 °N, 118.44 °W Los Angeles, ...'}, {'url': 'https://www.timeanddate.com/weather/@z-us-90016/hourly', 'content': 'See weather overview Detailed Hourly Forecast — Next 24 hours Show weather on: Next 24 hours June 22, 2024 June 23, 2024 June 24, 2024 June 25, 2024 June 26, 2024 June 27, 2024 June 28, 2024'}]", name='tavily_search_results_json', tool_call_id='call_EtyP8RyLpKhfXchP8bGovHMv')]}
{'messages': [HumanMessage(content='Whats the weather in LA?', id='f849bee2-8346-45f8-be50-e776397733ab'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 

In [34]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='f849bee2-8346-45f8-be50-e776397733ab'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EtyP8RyLpKhfXchP8bGovHMv', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-97ec91ac-4a92-4857-bc55-82831097eb09-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_EtyP8RyLpKhfXchP8bGovHMv'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}, next=('action',), config={'configurable': {'thread_id': '11', 'thread_ts': '1ef31527-48b2-6e26-8001-30b8efe6b535'}}, metadata={'source': 'loop', 'step': 1, 'writes

In [35]:
_id = to_replay.values['messages'][-1].tool_calls[0]['id']
to_replay.values['messages'][-1].tool_calls = [{'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in LA, accuweather'},
  'id': _id}]

In [36]:
branch_state = abot.graph.update_state(to_replay.config, to_replay.values)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='f849bee2-8346-45f8-be50-e776397733ab'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EtyP8RyLpKhfXchP8bGovHMv', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-97ec91ac-4a92-4857-bc55-82831097eb09-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_EtyP8RyLpKhfXchP8bGovHMv'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}


In [37]:
new_states = []
for state in abot.graph.get_state_history(thread):
    print(state)
    print('--')
    new_states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='f849bee2-8346-45f8-be50-e776397733ab'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EtyP8RyLpKhfXchP8bGovHMv', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-97ec91ac-4a92-4857-bc55-82831097eb09-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_EtyP8RyLpKhfXchP8bGovHMv'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}, next=('action',), config={'configurable': {'thread_id': '11', 'thread_ts': '1ef31555-b2ac-6128-8002-c82767192a43'}}, metadata={'source': 'update', 'ste

In [38]:
for event in abot.graph.stream(None, branch_state):
    for k, v in event.items():
        if k != "__end__":
            print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_EtyP8RyLpKhfXchP8bGovHMv'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 34.05, \'lon\': -118.24, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1719142862, \'localtime\': \'2024-06-23 4:41\'}, \'current\': {\'last_updated_epoch\': 1719142200, \'last_updated\': \'2024-06-23 04:30\', \'temp_c\': 17.2, \'temp_f\': 63.0, \'is_day\': 0, \'condition\': {\'text\': \'Clear\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/113.png\', \'code\': 1000}, \'wind_mph\': 3.8, \'wind_kph\': 6.1, \'wind_degree\': 280, \'wind_dir\': \'W\', \'pressure_mb\': 1011.0, \'pressure_in\': 29.86, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 87, \'cloud\': 0, \'feelslike_c\': 17.2, \'feel

In [39]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='f849bee2-8346-45f8-be50-e776397733ab'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EtyP8RyLpKhfXchP8bGovHMv', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-97ec91ac-4a92-4857-bc55-82831097eb09-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_EtyP8RyLpKhfXchP8bGovHMv'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}, next=('action',), config={'configurable': {'thread_id': '11', 'thread_ts': '1ef31527-48b2-6e26-8001-30b8efe6b535'}}, metadata={'source': 'loop', 'step'

In [40]:
_id = to_replay.values['messages'][-1].tool_calls[0]['id']

In [41]:
state_update = {"messages": [ToolMessage(
    tool_call_id=_id,
    name="tavily_search_results_json",
    content="54 degree celcius",
)]}

In [42]:
branch_and_add = abot.graph.update_state(
    to_replay.config, 
    state_update, 
    as_node="action")

In [43]:
for event in abot.graph.stream(None, branch_and_add):
    for k, v in event.items():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='f849bee2-8346-45f8-be50-e776397733ab'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EtyP8RyLpKhfXchP8bGovHMv', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-97ec91ac-4a92-4857-bc55-82831097eb09-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_EtyP8RyLpKhfXchP8bGovHMv'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173}), ToolMessage(content='54 degree celcius', name='tavily_search_results_json', id='8a8f6c9d-d87a-4ad3-aec3-bdc3599286f3', tool_call_id='call_EtyP8RyLpKhfXchP8bGovHMv'), AIMessage(content='T

In [44]:
from dotenv import load_dotenv

_ = load_dotenv()

In [45]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langgraph.checkpoint.sqlite import SqliteSaver

In [46]:
class AgentState(TypedDict):
    lnode: str
    scratch: str
    count: Annotated[int, operator.add]

In [47]:
def node1(state: AgentState):
    print(f"node1, count:{state['count']}")
    return {"lnode": "node_1",
            "count": 1,
           }
def node2(state: AgentState):
    print(f"node2, count:{state['count']}")
    return {"lnode": "node_2",
            "count": 1,
           }

In [48]:
def should_continue(state):
    return state["count"] < 3

In [49]:
builder = StateGraph(AgentState)
builder.add_node("Node1", node1)
builder.add_node("Node2", node2)

builder.add_edge("Node1", "Node2")
builder.add_conditional_edges("Node2", 
                              should_continue, 
                              {True: "Node1", False: END})
builder.set_entry_point("Node1")

In [50]:
memory = SqliteSaver.from_conn_string(":memory:")
graph = builder.compile(checkpointer=memory)

In [51]:
thread = {"configurable": {"thread_id": str(1)}}
graph.invoke({"count":0, "scratch":"hi"},thread)

node1, count:0
node2, count:1
node1, count:2
node2, count:3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

In [52]:
graph.get_state(thread)

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef315a4-f241-6208-8004-8987d1d08e8d'}}, metadata={'source': 'loop', 'step': 4, 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}}, created_at='2024-06-23T12:15:41.635630+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef315a4-f23c-6d70-8003-53d83d4bcf0c'}})

In [53]:
for state in graph.get_state_history(thread):
    print(state, "\n")

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef315a4-f241-6208-8004-8987d1d08e8d'}}, metadata={'source': 'loop', 'step': 4, 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}}, created_at='2024-06-23T12:15:41.635630+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef315a4-f23c-6d70-8003-53d83d4bcf0c'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 3}, next=('Node2',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef315a4-f23c-6d70-8003-53d83d4bcf0c'}}, metadata={'source': 'loop', 'step': 3, 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}}, created_at='2024-06-23T12:15:41.633873+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef315a4-f239-6008-8002-196fd27e4b9b'}}) 

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 2}, next=('Node1',), config={'configurable': {'thread_id': '1', 'thread_t

In [54]:
states = []
for state in graph.get_state_history(thread):
    states.append(state.config)
    print(state.config, state.values['count'])

{'configurable': {'thread_id': '1', 'thread_ts': '1ef315a4-f241-6208-8004-8987d1d08e8d'}} 4
{'configurable': {'thread_id': '1', 'thread_ts': '1ef315a4-f23c-6d70-8003-53d83d4bcf0c'}} 3
{'configurable': {'thread_id': '1', 'thread_ts': '1ef315a4-f239-6008-8002-196fd27e4b9b'}} 2
{'configurable': {'thread_id': '1', 'thread_ts': '1ef315a4-f231-6a4c-8001-d1f83b9eca20'}} 1
{'configurable': {'thread_id': '1', 'thread_ts': '1ef315a4-f22b-68f4-8000-1e0ca26db8d0'}} 0
{'configurable': {'thread_id': '1', 'thread_ts': '1ef315a4-f227-6ede-bfff-ced97af19e30'}} 0


In [55]:
states[-3]

{'configurable': {'thread_id': '1',
  'thread_ts': '1ef315a4-f231-6a4c-8001-d1f83b9eca20'}}

In [56]:
graph.get_state(states[-3])

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 1}, next=('Node2',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef315a4-f231-6a4c-8001-d1f83b9eca20'}}, metadata={'source': 'loop', 'step': 1, 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}}, created_at='2024-06-23T12:15:41.629288+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef315a4-f22b-68f4-8000-1e0ca26db8d0'}})

In [57]:
graph.invoke(None, states[-3])

node2, count:1
node1, count:2
node2, count:3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

In [58]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state.config, state.values['count'])

{'configurable': {'thread_id': '1', 'thread_ts': '1ef3163e-2d00-6530-8004-5a1588e2495b'}} 4
{'configurable': {'thread_id': '1', 'thread_ts': '1ef3163e-2cfb-6652-8003-ccc32e3d328b'}} 3
{'configurable': {'thread_id': '1', 'thread_ts': '1ef3163e-2cf6-6b0c-8002-254994553101'}} 2
{'configurable': {'thread_id': '1', 'thread_ts': '1ef315a4-f241-6208-8004-8987d1d08e8d'}} 4
{'configurable': {'thread_id': '1', 'thread_ts': '1ef315a4-f23c-6d70-8003-53d83d4bcf0c'}} 3
{'configurable': {'thread_id': '1', 'thread_ts': '1ef315a4-f239-6008-8002-196fd27e4b9b'}} 2
{'configurable': {'thread_id': '1', 'thread_ts': '1ef315a4-f231-6a4c-8001-d1f83b9eca20'}} 1
{'configurable': {'thread_id': '1', 'thread_ts': '1ef315a4-f22b-68f4-8000-1e0ca26db8d0'}} 0
{'configurable': {'thread_id': '1', 'thread_ts': '1ef315a4-f227-6ede-bfff-ced97af19e30'}} 0


In [59]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state,"\n")

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef3163e-2d00-6530-8004-5a1588e2495b'}}, metadata={'source': 'loop', 'step': 4, 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}}, created_at='2024-06-23T13:24:14.858155+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef3163e-2cfb-6652-8003-ccc32e3d328b'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 3}, next=('Node2',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef3163e-2cfb-6652-8003-ccc32e3d328b'}}, metadata={'source': 'loop', 'step': 3, 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}}, created_at='2024-06-23T13:24:14.856138+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef3163e-2cf6-6b0c-8002-254994553101'}}) 

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 2}, next=('Node1',), config={'configurable': {'thread_id': '1', 'thread_t

In [60]:
thread2 = {"configurable": {"thread_id": str(2)}}
graph.invoke({"count":0, "scratch":"hi"},thread2)

node1, count:0
node2, count:1
node1, count:2
node2, count:3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

In [62]:
states2 = []
for state in graph.get_state_history(thread2):
    states2.append(state.config)
    print(state.config, state.values['count'])   

{'configurable': {'thread_id': '2', 'thread_ts': '1ef31646-336f-60f8-8004-796fa5fcb781'}} 4
{'configurable': {'thread_id': '2', 'thread_ts': '1ef31646-3369-6a5e-8003-b1e5d4af2de6'}} 3
{'configurable': {'thread_id': '2', 'thread_ts': '1ef31646-3365-6d64-8002-b588e9a6b4b2'}} 2
{'configurable': {'thread_id': '2', 'thread_ts': '1ef31646-335f-6842-8001-2a1ac974b032'}} 1
{'configurable': {'thread_id': '2', 'thread_ts': '1ef31646-335a-6298-8000-5705f0fa13b6'}} 0
{'configurable': {'thread_id': '2', 'thread_ts': '1ef31646-3356-6210-bfff-f9e9d385547b'}} 0


In [63]:
save_state = graph.get_state(states2[-3])
save_state

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 1}, next=('Node2',), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef31646-335f-6842-8001-2a1ac974b032'}}, metadata={'source': 'loop', 'step': 1, 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}}, created_at='2024-06-23T13:27:50.274655+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef31646-335a-6298-8000-5705f0fa13b6'}})

In [64]:
save_state.values["count"] = -3
save_state.values["scratch"] = "hello"
save_state

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hello', 'count': -3}, next=('Node2',), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef31646-335f-6842-8001-2a1ac974b032'}}, metadata={'source': 'loop', 'step': 1, 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}}, created_at='2024-06-23T13:27:50.274655+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef31646-335a-6298-8000-5705f0fa13b6'}})

In [65]:
graph.update_state(thread2,save_state.values)

{'configurable': {'thread_id': '2',
  'thread_ts': '1ef31656-dd8a-67d0-8005-2280206e053f'}}

In [66]:
for i, state in enumerate(graph.get_state_history(thread2)):
    print(state, '\n')

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hello', 'count': 1}, next=('Node1',), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef31656-dd8a-67d0-8005-2280206e053f'}}, metadata={'source': 'update', 'step': 5, 'writes': {'Node2': {'lnode': 'node_1', 'scratch': 'hello', 'count': -3}}}, created_at='2024-06-23T13:35:17.614786+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef31646-336f-60f8-8004-796fa5fcb781'}}) 

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef31646-336f-60f8-8004-796fa5fcb781'}}, metadata={'source': 'loop', 'step': 4, 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}}, created_at='2024-06-23T13:27:50.281029+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef31646-3369-6a5e-8003-b1e5d4af2de6'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 3}, next=('Node2',), config={'configurable': {'

In [67]:
graph.update_state(thread2,save_state.values, as_node="Node1")

{'configurable': {'thread_id': '2',
  'thread_ts': '1ef31660-eabe-6d16-8006-1ef4322ccaad'}}

In [68]:
for i, state in enumerate(graph.get_state_history(thread2)):
    print(state, '\n')

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hello', 'count': -2}, next=('Node2',), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef31660-eabe-6d16-8006-1ef4322ccaad'}}, metadata={'source': 'update', 'step': 6, 'writes': {'Node1': {'lnode': 'node_1', 'scratch': 'hello', 'count': -3}}}, created_at='2024-06-23T13:39:47.434823+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef31656-dd8a-67d0-8005-2280206e053f'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hello', 'count': 1}, next=('Node1',), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef31656-dd8a-67d0-8005-2280206e053f'}}, metadata={'source': 'update', 'step': 5, 'writes': {'Node2': {'lnode': 'node_1', 'scratch': 'hello', 'count': -3}}}, created_at='2024-06-23T13:35:17.614786+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef31646-336f-60f8-8004-796fa5fcb781'}}) 

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(),

In [69]:
graph.invoke(None,thread2)

node2, count:-2
node1, count:-1
node2, count:0
node1, count:1
node2, count:2


{'lnode': 'node_2', 'scratch': 'hello', 'count': 3}

In [70]:
for state in graph.get_state_history(thread2):
    print(state,"\n")

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hello', 'count': 3}, next=(), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef31662-d35c-6e64-800b-ab30205d4d1e'}}, metadata={'source': 'loop', 'step': 11, 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}}, created_at='2024-06-23T13:40:38.670075+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef31662-d357-6e82-800a-9d9968aa259b'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hello', 'count': 2}, next=('Node2',), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef31662-d357-6e82-800a-9d9968aa259b'}}, metadata={'source': 'loop', 'step': 10, 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}}, created_at='2024-06-23T13:40:38.668036+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef31662-d351-67e4-8009-baae28394b1c'}}) 

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hello', 'count': 1}, next=('Node1',), config={'configurable': {'thread_id': '2'